In [1]:
using Revise
using Dates
using Distributions
using Random
using DataFrames
using CSV
using Plots
using StatsBase
using LinearAlgebra
using Statistics
import Pkg
Pkg.activate("..")
Pkg.instantiate()

using SequentialNorta

  Activating project at `c:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\SPCM\src\scenario_generation\sequential_norta`
Precompiling project...
  ✓ SequentialNorta
  1 dependency successfully precompiled in 10 seconds. 287 already precompiled.


In [2]:
scen_generator = scenario_generator_init()
# Save the objects from scen_generator into individual variables
unique_forecast_times = scen_generator["unique_forecast_times"]
unique_issue_times = scen_generator["unique_issue_times"]
start_date = scen_generator["start_date"]
corr_forecast_issue_times = scen_generator["corr_forecast_issue_times"]
forecast_scenario_length = scen_generator["forecast_scenario_length"]
number_of_scenarios = scen_generator["number_of_scenarios"]
issue_idcs = scen_generator["issue_idcs"]
lp_load = scen_generator["lp_load"]
lp_solar = scen_generator["lp_solar"]
lp_wind = scen_generator["lp_wind"]
M_load = scen_generator["M_load"]
M_solar = scen_generator["M_solar"]
M_wind = scen_generator["M_wind"]
load_marginals_by_issue = scen_generator["load_marginals_by_issue"]
solar_marginals_by_issue = scen_generator["solar_marginals_by_issue"]
wind_marginals_by_issue = scen_generator["wind_marginals_by_issue"]
load_landing_probabilities = scen_generator["load_landing_probabilities"]
solar_landing_probabilities = scen_generator["solar_landing_probabilities"]
wind_landing_probabilities = scen_generator["wind_landing_probabilities"]
load_actual_avg = scen_generator["load_actual_avg"]
solar_actual_avg = scen_generator["solar_actual_avg"]
wind_actual_avg = scen_generator["wind_actual_avg"]
sunny_decision_hours = scen_generator["sunny_decision_hours"]
load_actual_avg_GW = scen_generator["load_actual_avg_GW"]
solar_actual_avg_cf = scen_generator["solar_actual_avg_cf"]
wind_actual_avg_cf = scen_generator["wind_actual_avg_cf"]
decision_mdl_lkd_length = scen_generator["decision_mdl_lkd_length"];

In [3]:
forecast2model_indices = findall(in(unique_forecast_times), load_actual_avg[!, :time_index])

8736-element Vector{Int64}:
    7
    8
    9
   10
   11
   12
   13
   14
   15
   16
    ⋮
 8734
 8735
 8736
 8737
 8738
 8739
 8740
 8741
 8742

In [4]:
data = lp_solar
issue_idcs = issue_idcs
dec_mdl_lkd_len = decision_mdl_lkd_length
is_solar = true

true

In [6]:
lp_solar

363×48 transpose(::Matrix{Float64}) with eltype Float64:
 0.99  0.99  0.99  0.99  0.99  0.99  …  0.99  0.99  0.99  0.99  0.99  0.99
 0.99  0.99  0.99  0.99  0.99  0.99     0.99  0.99  0.99  0.99  0.99  0.99
 0.99  0.99  0.99  0.99  0.99  0.99     0.99  0.99  0.99  0.99  0.99  0.99
 0.99  0.99  0.99  0.99  0.99  0.99     0.99  0.99  0.99  0.99  0.99  0.99
 0.99  0.99  0.99  0.99  0.99  0.99     0.99  0.99  0.99  0.99  0.99  0.99
 0.99  0.99  0.99  0.99  0.99  0.99  …  0.99  0.99  0.99  0.99  0.99  0.99
 0.99  0.99  0.99  0.99  0.99  0.99     0.99  0.99  0.99  0.99  0.99  0.99
 0.99  0.99  0.99  0.99  0.99  0.99     0.99  0.99  0.99  0.99  0.99  0.99
 0.99  0.99  0.99  0.99  0.99  0.99     0.99  0.99  0.99  0.99  0.99  0.99
 0.99  0.99  0.99  0.99  0.99  0.99     0.99  0.99  0.99  0.99  0.99  0.99
 ⋮                             ⋮     ⋱                    ⋮           
 0.99  0.99  0.99  0.99  0.99  0.99     0.99  0.99  0.99  0.99  0.99  0.99
 0.99  0.99  0.99  0.99  0.99  0.99  …  0.99  0

In [7]:
x = copy(data)
allequal_set = Set(findall(allequal, eachcol(x)));
allequal_ind = sort(collect(allequal_set));
allindex_set = Set(collect(1:48));
alldifferent_ind = sort(collect(setdiff(allindex_set, allequal_set))); # Index for columns whose st. dev. isn't zero
x_upd = x[:, alldifferent_ind];

In [8]:
solar_dec_ts_df = solar_actual_avg[forecast2model_indices, :]

Row,time_index,avg_actual
,DateTime,Float64
1,2018-01-01T00:00:00,0.0
2,2018-01-01T01:00:00,0.0
3,2018-01-01T02:00:00,0.0
4,2018-01-01T03:00:00,0.0
5,2018-01-01T04:00:00,0.0
6,2018-01-01T05:00:00,0.0
7,2018-01-01T06:00:00,0.0
8,2018-01-01T07:00:00,6.91694
9,2018-01-01T08:00:00,135.23


In [9]:
sunny_decision_hours

5106-element Vector{Int64}:
    7
    8
    9
   10
   11
   12
   13
   14
   15
   16
    ⋮
 8728
 8729
 8730
 8731
 8732
 8733
 8734
 8735
 8736

In [10]:
# get the indices where solar_dec_ts_df is non-zero
non_zero_indices = findall(solar_dec_ts_df[!, :avg_actual] .!= 0)

4880-element Vector{Int64}:
    8
    9
   10
   11
   12
   13
   14
   15
   16
   17
    ⋮
 8723
 8724
 8725
 8726
 8727
 8728
 8729
 8730
 8731

In [11]:
sunny_decision_hours == non_zero_indices

false

In [12]:
differences = setdiff(non_zero_indices, sunny_decision_hours)
println("Differences: ", differences)

Differences: [2574, 2598, 2622, 2646, 2670, 2694, 2718, 2742, 2766, 2790, 2814, 2838, 2862, 2886, 2910, 2934, 2958, 2982, 3006, 3030, 3054, 3078, 3102, 3126, 3150, 3174, 3198, 3222, 3246, 3270, 3294, 3318, 3342, 3366, 3390, 3414, 3438, 3462, 3486, 3510, 3534, 3558, 3582, 3606, 3630, 3654, 3678, 3702, 3726, 3750, 3774, 3798, 3822, 3846, 3870, 3894, 3918, 3942, 3966, 3990, 4014, 4038, 4062, 4086, 4110, 4134, 4158, 4182, 4206, 4230, 4254, 4278, 4302, 4326, 4350, 4374, 4398, 4422, 4446, 4470, 4494, 4518, 4542, 4566, 4590, 4614, 4638, 4662, 4686, 4710, 4734, 4758, 4782, 4806, 4830, 4854, 4878, 4902, 4926, 4950, 4974, 4998, 5022, 5046, 5070, 5094, 5118, 5142, 5166, 5190, 5214, 5238, 5262, 5286, 5310, 5334, 5358, 5382, 5406, 5430, 5454, 5478, 5502, 5526, 5550]


In [13]:
issubset(non_zero_indices, sunny_decision_hours)

false

In [14]:
not_in_sunny_hours = setdiff(non_zero_indices, sunny_decision_hours)
println("Indices not in sunny_decision_hours:")
for idx in not_in_sunny_hours
    println(idx)
end

Indices not in sunny_decision_hours:
2574
2598
2622
2646
2670
2694
2718
2742
2766
2790
2814
2838
2862
2886
2910
2934
2958
2982
3006
3030
3054
3078
3102
3126
3150
3174
3198
3222
3246
3270
3294
3318
3342
3366
3390
3414
3438
3462
3486
3510
3534
3558
3582
3606
3630
3654
3678
3702
3726
3750
3774
3798
3822
3846
3870
3894
3918
3942
3966
3990
4014
4038
4062
4086
4110
4134
4158
4182
4206
4230
4254
4278
4302
4326
4350
4374
4398
4422
4446
4470
4494
4518
4542
4566
4590
4614
4638
4662
4686
4710
4734
4758
4782
4806
4830
4854
4878
4902
4926
4950
4974
4998
5022
5046
5070
5094
5118
5142
5166
5190
5214
5238
5262
5286
5310
5334
5358
5382
5406
5430
5454
5478
5502
5526
5550


In [15]:
step_differences = diff(differences)
println("Step differences: ", step_differences)

Step differences: [24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24]


In [16]:
x_upd

363×28 Matrix{Float64}:
 0.99  0.99  0.99      0.868687  …  0.343434   0.232323   0.99  0.99
 0.99  0.99  0.01      0.01         0.171717   0.10101    0.99  0.99
 0.99  0.99  0.01      0.505051     0.131313   0.0707071  0.99  0.99
 0.99  0.99  0.848485  0.868687     0.232323   0.121212   0.99  0.99
 0.99  0.99  0.757576  0.656566     0.272727   0.151515   0.99  0.99
 0.99  0.99  0.01      0.01      …  0.505051   0.20202    0.99  0.99
 0.99  0.99  0.99      0.01         0.494949   0.212121   0.99  0.99
 0.99  0.99  0.575758  0.585859     0.0909091  0.141414   0.99  0.99
 0.99  0.99  0.01      0.01         0.0808081  0.151515   0.99  0.99
 0.99  0.99  0.01      0.01         0.363636   0.20202    0.99  0.99
 ⋮                               ⋱             ⋮                
 0.99  0.99  0.01      0.636364     0.222222   0.0909091  0.99  0.99
 0.99  0.99  0.01      0.01      …  0.272727   0.10101    0.99  0.99
 0.99  0.99  0.01      0.606061     0.525253   0.171717   0.99  0.99
 0.99  0.99  0

In [17]:
if ishermitian(cor(x_upd))
    Σ_Z = LinearAlgebra.cholesky(cor(x_upd));
else
    Σ_Z = factorize(cor(x_upd));
end
M = Σ_Z.L;

In [58]:

# initialize vector for indices of zeros
solar_zero_indices = Vector{Int64}();

allequal_ind_1d = allequal_ind[1:length(allequal_ind) ÷ 2];

# loop through the number issue times
for i in issue_idcs
    solar_zero_indices = vcat(solar_zero_indices, allequal_ind_1d .+ (i-1)*24)
end
# checks of lengths - both are TRUE
# println(dec_mdl_lkd_len == 363*24+24)
# println(363*48 == size(corr_forecast_issue_times,1))

solar_vector = ones(dec_mdl_lkd_len);
# for idx in solar_zero_indices
#     solar_vector[idx] = 0
# end
solar_vector[solar_zero_indices] .= 0;

all_decision_hours = Set(collect(1:dec_mdl_lkd_len));
sunny_decision_hours = sort(collect(setdiff(all_decision_hours, solar_zero_indices)));


In [19]:
solar_zero_indices

3630-element Vector{Int64}:
    1
    2
    3
    4
    5
    6
   21
   22
   23
   24
    ⋮
 8690
 8691
 8692
 8693
 8694
 8709
 8710
 8711
 8712

In [59]:
sunny_decision_hours

5469-element Vector{Int64}:
    6
    7
    8
    9
   10
   11
   12
   13
   14
   15
    ⋮
 8728
 8729
 8730
 8731
 8732
 8733
 8734
 8735
 8736

In [22]:
# verify that where solar_zero_indices are, that he solar_dec_ts_df is zero in :avg_actual
for idx in solar_zero_indices
    hour_idx = idx - 1
    if solar_dec_ts_df[hour_idx + 1, :avg_actual] != 0
        println(solar_dec_ts_df[hour_idx + 1, :avg_actual])
        println("Error at index: ", idx)
        println("Day of the year: ", Dates.dayofyear(start_date + Dates.Hour(hour_idx)))
    end
end

0.012837500000000002
Error at index: 2574
Day of the year: 108
0.10270000000000001
Error at index: 2598
Day of the year: 109
0.07394816666666666
Error at index: 2622
Day of the year: 110
0.03361
Error at index: 2646
Day of the year: 111
0.15426291666666667
Error at index: 2670
Day of the year: 112
0.41625500000000004
Error at index: 2694
Day of the year: 113
0.48291816666666665
Error at index: 2718
Day of the year: 114
0.42666541666666663
Error at index: 2742
Day of the year: 115
1.0253154166666667
Error at index: 2766
Day of the year: 116
1.1746949999999998
Error at index: 2790
Day of the year: 117
1.4401754166666667
Error at index: 2814
Day of the year: 118
2.1231381666666667
Error at index: 2838
Day of the year: 119
2.716945416666667
Error at index: 2862
Day of the year: 120
1.3874147083333332
Error at index: 2886
Day of the year: 121
0.19481608333333333
Error at index: 2910
Day of the year: 122
0.1754125
Error at index: 2934
Day of the year: 123
0.109375
Error at index: 2958
Day of

In [21]:
72*24

1728

Ok, I suspect something is wrong with sunny_decision_hours

In [23]:
allequal_ind_1d


10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
 21
 22
 23
 24

So these are the columns where all the solar output should be zero but is that true?

In [28]:
zero_indices = Vector{Int64}()

for idx in allequal_ind_1d
    for i in 0:363
        check_idx = idx + i * 24
        if solar_dec_ts_df[check_idx, :avg_actual] != 0
            println("Error at index: ", check_idx)
            println("Value: ", solar_dec_ts_df[check_idx, :avg_actual])
        else
            push!(zero_indices, check_idx)
        end
    end
end

Error at index: 2574
Value: 0.012837500000000002
Error at index: 2598
Value: 0.10270000000000001
Error at index: 2622
Value: 0.07394816666666666
Error at index: 2646
Value: 0.03361
Error at index: 2670
Value: 0.15426291666666667
Error at index: 2694
Value: 0.41625500000000004
Error at index: 2718
Value: 0.48291816666666665
Error at index: 2742
Value: 0.42666541666666663
Error at index: 2766
Value: 1.0253154166666667
Error at index: 2790
Value: 1.1746949999999998
Error at index: 2814
Value: 1.4401754166666667
Error at index: 2838
Value: 2.1231381666666667
Error at index: 2862
Value: 2.716945416666667
Error at index: 2886
Value: 1.3874147083333332
Error at index: 2910
Value: 0.19481608333333333
Error at index: 2934
Value: 0.1754125
Error at index: 2958
Value: 0.109375
Error at index: 2982
Value: 2.5447700833333333
Error at index: 3006
Value: 5.583255583333333
Error at index: 3030
Value: 5.875557875000001
Error at index: 3054
Value: 6.069577083333333
Error at index: 3078
Value: 4.28059425

In [32]:
# find the max value from solar_dec_ts_df in :avg_actual
max_value = maximum(solar_dec_ts_df[!, :avg_actual])
avg_value = mean(solar_dec_ts_df[!, :avg_actual])
# find the 1 percentile value
percentile_1 = quantile(solar_dec_ts_df[!, :avg_actual], 0.01)
percentile_2 = quantile(solar_dec_ts_df[!, :avg_actual], 0.02)

0.0

What if I put the actuals into a matrix form and did the all same proces?

In [33]:
363*48

17424

In [35]:
print(size(corr_forecast_issue_times))

(17424, 2)

In [45]:
solar_dec_ts = solar_actual_avg[forecast2model_indices, :]

Row,time_index,avg_actual
,DateTime,Float64
1,2018-01-01T00:00:00,0.0
2,2018-01-01T01:00:00,0.0
3,2018-01-01T02:00:00,0.0
4,2018-01-01T03:00:00,0.0
5,2018-01-01T04:00:00,0.0
6,2018-01-01T05:00:00,0.0
7,2018-01-01T06:00:00,0.0
8,2018-01-01T07:00:00,6.91694
9,2018-01-01T08:00:00,135.23


In [62]:
solar_dec_ts

Row,time_index,avg_actual
,DateTime,Float64
1,2018-01-01T00:00:00,0.0
2,2018-01-01T01:00:00,0.0
3,2018-01-01T02:00:00,0.0
4,2018-01-01T03:00:00,0.0
5,2018-01-01T04:00:00,0.0
6,2018-01-01T05:00:00,0.0
7,2018-01-01T06:00:00,0.0
8,2018-01-01T07:00:00,6.91694
9,2018-01-01T08:00:00,135.23


In [52]:
# Organize solar_actual_avg so that every 48 hours are in a row,
# and each row repeats the previous second half of the previous row, except the first row.
# There will be 363 rows.

solar_matrix = zeros(363, 48)
for i in 1:363
    if i == 1
        solar_matrix[i, :] .= solar_dec_ts[1:48, :avg_actual]
    else
        solar_matrix[i, 1:24] .= solar_matrix[i-1, 25:48]
        solar_matrix[i, 25:48] .= solar_dec_ts[(24*(i-1) + 1):(24*i), :avg_actual]
    end
end


In [47]:
48*(363-1) + 25

17401

In [48]:
24*(363-1) + 25

8713

In [51]:
(24*(362-1) + 1):(24*362)

8665:8688

In [50]:
(24*(363-1) + 1):(24*363)

8689:8712

In [53]:
solar_matrix

363×48 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.018     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.018     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.001125  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0045    0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.028125  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.041625  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0495    0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.032625  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.068625  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.09675   0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮         ⋱                      ⋮         
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0       0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0       0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0 

In [54]:
x = copy(solar_matrix)
allequal_set = Set(findall(allequal, eachcol(x)));
allequal_ind = sort(collect(allequal_set));
allindex_set = Set(collect(1:48));
alldifferent_ind = sort(collect(setdiff(allindex_set, allequal_set))); # Index for columns whose st. dev. isn't zero
x_upd = x[:, alldifferent_ind];

In [64]:
alldifferent_ind

30-element Vector{Int64}:
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
  ⋮
 36
 37
 38
 39
 40
 41
 42
 43
 44

In [55]:
x_upd

363×30 Matrix{Float64}:
 0.0  0.0   6.91694  135.23    345.787   …  182.753   42.9719   0.018     0.0
 0.0  0.0   7.52672  132.929    73.5261     182.753   42.9719   0.018     0.0
 0.0  0.0   7.52672  132.929    73.5261     376.521   73.4109   0.001125  0.0
 0.0  0.0  11.6441   204.791   480.449      150.255   14.7394   0.0045    0.0
 0.0  0.0  19.2432   221.793   468.623      340.126   66.9729   0.028125  0.0
 0.0  0.0   4.59919  154.795   380.23    …  278.534   55.5939   0.041625  0.0
 0.0  0.0  19.4889   200.795   349.606      164.401   43.8404   0.0495    0.0
 0.0  0.0   1.17991   14.334    52.6993     320.166   72.2056   0.032625  0.0
 0.0  0.0   9.10436  147.562   321.968      253.652   67.5596   0.068625  0.0
 0.0  0.0   3.0949    97.8714  229.354      138.537   25.3429   0.09675   0.0
 ⋮                                       ⋱                                
 0.0  0.0  25.9218   263.514   476.343      320.147   44.5153   0.0       0.0
 0.0  0.0  34.4267   297.39    503.967   … 

In [61]:
if ishermitian(cor(x_upd))
    Σ_Z = LinearAlgebra.cholesky(cor(x_upd));
else
    Σ_Z = factorize(cor(x_upd));
end
M = Σ_Z.L;